In [ ]:
! pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=5edd714878101ea254c75c96b8c226baeb155fbaa2f1f240b237648b5983d8b4
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
dataset_dir = "../content/"
glove_dir = './glove.6B/'
save_dir = './content/'

In [ ]:
df = pd.read_csv(os.path.join(dataset_dir, 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')
y = df['domain1_score']
df = df.dropna(axis=1)
df = df.drop(columns=['rater1_domain1', 'rater2_domain1'])

df

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8
...,...,...,...,...
12971,21626,8,In most stories mothers and daughters are eit...,35
12972,21628,8,I never understood the meaning laughter is th...,32
12973,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",40
12974,21630,8,Trippin' on fen...,40


In [ ]:
min_scores = np.array([-1, 2, 1, 0, 0, 0, 0, 0, 0])
max_scores = np.array([-1, 12, 6, 3, 3, 4, 4, 30, 60])

In [ ]:
old_minimum = min_scores[df['essay_set']]
old_maximum = max_scores[df['essay_set']]
old_range = old_maximum - old_minimum
new_minimum = 0
new_maximum = 100
new_range = (new_maximum - new_minimum)  
df['score'] = (((df['domain1_score'] - old_minimum) * new_range) / old_range) + new_minimum

y = np.round(df['score'])

df

,essay_id,essay_set,essay,domain1_score,score
0,1,1,"Dear local newspaper, I think effects computer...",8,60.000000
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,70.000000
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,50.000000
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,80.000000
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,60.000000
...,...,...,...,...,...
12971,21626,8,In most stories mothers and daughters are eit...,35,58.333333
12972,21628,8,I never understood the meaning laughter is th...,32,53.333333
12973,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",40,66.666667
12974,21630,8,Trippin' on fen...,40,66.666667


In [ ]:
y

0        60.0
1        70.0
2        50.0
3        80.0
4        60.0
         ... 
12971    58.0
12972    53.0
12973    67.0
12974    67.0
12975    67.0
Name: score, Length: 12976, dtype: float64

In [ ]:
import numpy as np
from gensim.models import Word2Vec
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

def esai_ke_listkata(essay_v, remove_stopwords):
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    kata = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        kata = [w for w in kata if not w in stops]
    return (kata)

def esai_ke_kalimat(essay_v, remove_stopwords):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    kalimat = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            kalimat.append(esai_ke_listkata(raw_sentence, remove_stopwords))
    return kalimat

def makeFeatureVec(kata, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    for word in kata:
        if word in model:
            num_words += 1
            featureVec = np.add(featureVec, model[word])       
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [34]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from tensorflow.keras.models import Sequential, load_model, model_from_config
import tensorflow.keras.backend as K

def getmodel():
    model = Sequential()
    model.add(LSTM(200, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 200], return_sequences=True))
    model.add(LSTM(128, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [35]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

corpus = []
for essay in df['essay']:
    corpus.append(esai_ke_listkata(essay, True))

embedding_dict={}
with open('/content/glove.6B.200d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:],'float32')
        embedding_dict[word] = vectors

cv = KFold(n_splits=5, shuffle=True)
results = []
y_pred_list = []

count = 1
for traincv, testcv in cv.split(X):
    
    print("\n--------Fold {}--------\n".format(count))
    X_test, X_train, y_test, y_train = df.iloc[testcv], df.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    sentences = []
    
    for essay in train_essays:
        sentences += esai_ke_listkata(essay, remove_stopwords = True)

    num_features = 200 

    model = embedding_dict

    clean_train_essays = []
    for essay_v in train_essays:
        clean_train_essays.append(esai_ke_listkata(essay_v, remove_stopwords=True))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_v in test_essays:
        clean_test_essays.append(esai_ke_listkata( essay_v, remove_stopwords=True ))
    testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    lstm_model = getmodel()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
    y_pred = lstm_model.predict(testDataVecs)

    if count == 5:
         lstm_model.save('./content/model_lstm.h5')

    y_pred = np.round(y_pred)
    
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Skor Kappa: {}".format(result))
    results.append(result)

    count += 1


--------Fold 1--------

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 1, 200)            320800    
                                                                 
 lstm_19 (LSTM)              (None, 128)               168448    
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 489,377
Trainable params: 489,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
163/163 [==============================] - 11s 30ms/step - loss: 2121.8135 - mae: 40.4628
Epoch 2/50
163/163 [==============================] - 

In [36]:
print("Rata-rata skor kappa: ", np.around(np.array(results).mean(),decimals=4))

Rata-rata skor kappa:  0.6397


In [37]:
import math
from gensim.test.utils import datapath

contoh_konten = """
        I aegre waf the evansmant ov tnachnolage. The evansmant ov tnachnolige is being to halp fined a kohar froi alnsas. Tnanchnolage waf ont ot we wod not go to the moon. Tnachnologe evans as we maech at. The people are in tnacholege to the frchr fror the good ov live. Famas invanyor ues tnacholage leki lena orde dvanse and his fling mashine. Tnachologe is the grat
    """ 

content = contoh_konten
    
if len(content) > 20:
    num_features = 200
    clean_test_essays = []
    clean_test_essays.append(esai_ke_listkata( content, remove_stopwords=True ))
    testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    testDataVecs = np.array(testDataVecs)
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))

    prediksi = lstm_model.predict(testDataVecs)

    if math.isnan(prediksi):
        prediksi = 0
    else:
      prediksi = np.round(prediksi)

    if prediksi < 0:
        prediksi = 0
else:
    prediksi = 0
    
print(prediksi)

1/1 [==============================] - 0s 43ms/step
[[38.]]
